<center><font size=6><b>GEOS 669 Geodetic Methods and Modeling</b></font></center>

# Lab 6: InSAR Timeseries Analysis with HyP3 & MintPY


**Assignment Due Date**: October 13th, 2021

Your deliverable for this lab will be this notebook, with additional sections ** and images** as outlined below. Please rename the notebook from `GEOS669-Lab6.ipynb` to `<last_name>_GEOS669-Lab6.ipynb`. For example, I (Emily Graves) would rename the notebook to `Graves_GEOS669-Lab6.ipynb`. You can do this by clicking on the filename in the top-left corner of this window (similar to, e.g., Google Docs). Download the completed file and images by checking its box in the JupyterHub file browser and selecting "Download" from the menu that appears or in the document's "File" menu under "Download as". **Then submit a zip file with the notebook and images to Canvas under the Lab 6 assignment no later than 1pm October 13th**.

### Introduction

Complete shift in gears now! We're moving from GPS processing into InSAR processing. The goal for today is to get you comfortable finding you way around the Alaska Satellite Facility's data portal, [Vertex](https://search.asf.alaska.edu/#/), using their free processing service, moving those data into opensarlab and performing a timeseries analysis on these data using [MintPy](https://github.com/insarlab/MintPy).

A note of warning: there's a lot of vocabulary flying (ha!) around in InSAR processing. Most of these are standard satellite related terms. If there's something you don't understand, it's usually a good idea to search for the term, the satellite, and read up on it.



### Vertex Account

For this weeks lab, you will need to set up a Vertex account. Please have this ready before lab. To make your account, go to the [Vertex website](https://search.asf.alaska.edu/#/) and on the upper right click "Sign In". In the new window, scroll down and select "Register". Fill in the necessary fields and select "Register for EarthData Login".

### Step 1: Find an Event

I would like for you to find an interesting event (earthquake, volcanic eruption, etc). It doesn't so much matter what the signal is as it does that it can be explored using InSAR, ideally this is large amplitude, in an arid region. This can or cannot be related to your graduate research projects. If you have an event in mind, find the time and location.
 

### Step 2: Data Selection

In Vertex, the top search tool bar allows you to set which dataset or satellite you search as well as an Area of Interest and additional filters for time and data type. If you are looking at a recent event (post-2014), leave the defaults in the search bar. On the map, zoom and pan to your Area of Interest (AOI). Use the click and drag mouse tool to define an AOI on the map. Select "Search" in the top search tool bar. One or more blue boxes will appear overlapping your AOI. These are Sentinel-1 SAR aquisition frames. Select a frame that completely covers your AOI (it should turn from red to blue). A bottom data window outlines the data associated with the selected frame. The "Scene Detail" menu provides satellite information such as the path and frame number, polarization, beam mode, and flight direction.

Select "SBAS" from the bottom search tool. A new SBAS menu should appear. The number of pairs that will be generated is listed in the upper left corner. This is important. We would like to have around 100 pairs. 

On the right, a baseline image shows the SAR aquisitions plotted over time and baseline. The connecting black lines indicate pre-specified pairs. You can manually add additional pairs using the "Custom Pair" tool. Select the "+" and click on two SAR data points. A dotted line should appear between the two points. Add a few pairs to process that were not automatically generated. The number of pairs on the left should increase. Since this is set up as short baseline processing, add a few seasonal pairs that span from one year to another. In regions of low coherence, such as the northern regions with snowfall, seasonal pairs are necesary to determine deformation. 

You can adjust the number of pairs by sliding the baseline bar of the left of the image to adjust the pairs allowed baseline difference and the meter at the bottom of the days between aquisitions. You can set a dates filter under "SBAS Criteria".

### Step 3: Start Processing

Once you have the pairs you want, select the "Add all results to On Demand Queue" button next to the number of pairs. It looks like three stacked windows. Under "InSAR GAMMA", click "Add * SLC Pairs", where * is the number of pairs.

In the top menu, select the "On Demand" button, and "On Demand Queue". A new menu will appear where you can choose what products are produced in addition to the unwrapped phase images. **Select "Include DEM", "Include Inc. Angle Map","Include LOS Displacement", and "Include Wrapped phase"**. Click "Submit * Jobs", and provide a **meaningful** project name (this will come back later!).

**The processing will take awhile depending on the number of pairs.**



## Setting up MintPy on OSL

We've gotta do a number of things now to get OSL MintPy ready. You're at the forefront as this is still under development. 

### New conda environment

Run the code cell below to set up a new conda environment `insar_analysis` that contains all the packages necessary for MintPy.

In [ ]:
%%javascript
var kernel = Jupyter.notebook.kernel;
var command = ["notebookUrl = ",
               "'", window.location, "'" ].join('')
kernel.execute(command)

In [ ]:
import glob
import ipywidgets
import os
import fileinput

from IPython.display import Markdown, display

In [ ]:
%%bash
# create backups if needed
if [ ! -f /home/jovyan/conda_environments/.git/config.save ]; then
    echo "back up of conda config"
    cp  /home/jovyan/conda_environments/.git/config /home/jovyan/conda_environments/.git/config.save
fi

if [ ! -f /home/jovyan/notebooks/.git/config.save ]; then
    echo "backup notebook config"
    cp /home/jovyan/notebooks/.git/config /home/jovyan/notebooks/.git/config.save
fi

#edit the config files
echo "editing the config files"
sed 's|fetch = +refs/heads/main:refs/remotes/origin/main|fetch = +refs/heads/*:refs/remotes/origin/*|g' < /home/jovyan/conda_environments/.git/config.save > /home/jovyan/conda_environments/.git/config
sed 's|fetch = +refs/heads/main:refs/remotes/origin/main|fetch = +refs/heads/*:refs/remotes/origin/*|g' < /home/jovyan/notebooks/.git/config.save > /home/jovyan/notebooks/.git/config
sed 's|fetch = +refs/heads/master:refs/remotes/origin/master|fetch = +refs/heads/*:refs/remotes/origin/*|g' < /home/jovyan/notebooks/.git/config.save > /home/jovyan/notebooks/.git/config


#getting the mintpy environment
echo "getting mintpy environment ..."
cd /home/jovyan/conda_environments
git fetch --all
git checkout mintpy

echo "getting mintpy notebooks ..."
cd /home/jovyan/notebooks
git fetch --all
git checkout mintpy

#DO NOT CHANGE THIS VARIABLE!
name="insar_analysis"

echo "create the new environment insar_analysis"
conda env create -f /home/jovyan/conda_environments/Environment_Configs/${name}_env.yml --prefix /home/jovyan/.local/envs/${name} --force


### Local configuration

The steps below are necessary to make sure this will run, and installs a few more packages.

In [ ]:
%%bash
name="insar_analysis"

## ONLY IF THE COMMAND AT THE BOTTOM FAILS, UNCOMMENT THIS SECTION BELOW AND RERUN
#echo "channels:
#  - conda-forge
#  - defaults

#channel_priority: strict

#create_default_packages:
#  - jupyter
#  - kernda

#envs_dirs:
#  - /home/jovyan/.local/envs
#  - /opt/conda/envs" > ~/.condarc


echo "update ipykernel with env name and location"
conda run -n $name kernda --display-name $name -o /home/jovyan/.local/envs/${name}/share/jupyter/kernels/python3/kernel.json

cd ~/conda_environments 

source install_insar_analysis_pkgs.sh

conda clean -p -t --yes

### SETUP DONE!

## RUNNING THE TIMESERIES ANALYSIS

Now, with your environment in place, there are two other notebooks to execute:

* download data from vertex into OSL and prepare a mintpy stack
* run the mintpy processing

Execution of the code cells below will generate the respective links for you. **FOLLOW ALL OF THE INSTRUCTIONS IN THESE NOTEBOOKS! READ ALL THE COMMENTS IN THE CODE!**

In [ ]:
user = !echo $JUPYTERHUB_USER
display(Markdown(f'<text style=color:green> Open <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/notebooks/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb" target="_blank">Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb</a> to download data from vertext and pre</text>'))


For the time series analysis below, the download of the atmosphere product will take the longest time. Be prepared for a 30+ minute endeavor. Make sure your notebook won't idle out after 1 hr of inactivity. Please perform the analysis in the individual steps, not as a single process. Or you won't learn much.

In [ ]:
user = !echo $JUPYTERHUB_USER
display(Markdown(f'<text style=color:green> Open <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/notebooks/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb" target="_blank"> MintPy_Time_Series_From_Prepared_Data_Stack.ipynb </a>to run an InSAR time-series analysis on your data using MintPy</text>'))

## DELIVERABLES

In the directory with your data you should find a `MintPy` subfolder. The `pic` subfolder holds all the products created during the analysis. I think with the knowledge from the lectures a lot of these images should be intuitive. Explore!

In a text cell below, explain why you selected the region you picked and for your selected region include and discuss, in the following order, plots of:

- the perpendicular baseline history
- the coherence matrix
- the interferogram network
- plot for the average spatial coherence
- plot for the average temporal coherence
- inverted time series plot
- average velocity plot
- velocity error plot

In section 6 of the notebook you can display timeseries for individual regions of interest by clicking on them. Find an interesting region, save the image and include it here. Discuss what's going on.